In [71]:
import json
import utils
import pandas as pd

In [124]:
load_path = "setup_data/demo_json"

with open(load_path, "r", encoding="utf-8") as f:
    demo_json = json.load(f)

In [125]:
load_path = "setup_data/survey_json"

with open(load_path, "r", encoding="utf-8") as f:
    survey_json = json.load(f)

In [73]:
load_path = "output_data/json_answers"

with open(load_path, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df.shape

(1078, 36)

In [74]:
utils.check_extraction(demo_json, df)


=== SECTOR ===
                                                    Observed  Expected
clothing & footware                                 0.358071      0.37
electronics                                         0.050093      0.04
entertainment & leisure                             0.077922      0.09
food & beverage                                     0.081633      0.09
healt & beauty                                      0.099258      0.10
home & garden                                       0.090909      0.09
is not clothing & footwear, is not health & bea...  0.063080      0.00
jewellery & accessories                             0.072356      0.07
other                                               0.000000      0.06
sporting goods & athleisure                         0.106679      0.09

=== PRICE_POSITION ===
                                            Observed  Expected
is not luxury, is not upper, is not middle  0.260668      0.00
luxury                                      0.019481 

In [75]:
cat_list = list(demo_json.keys())
cat_list

['sector', 'price_position', 'worldwide_store_count']

In [102]:
df_stats = df[[c for c in df if ('Q' in c) or (c in cat_list)]].melt(
    id_vars=cat_list,
    var_name="Q",       # name of the former column labels (Q1..Q29)
    value_name="value"  # name of the numeric values
)

In [103]:
df_stats

,sector,price_position,worldwide_store_count,Q,value
0,clothing & footware,"is not luxury, is not upper, is not middle",500-1000,Q1,3
1,healt & beauty,middle,250-500,Q1,2
2,home & garden,middle,250-500,Q1,2
3,clothing & footware,upper,1000-3000,Q1,3
4,sporting goods & athleisure,upper,0-50,Q1,2
...,...,...,...,...,...
31257,clothing & footware,"is not luxury, is not upper, is not middle",5000-10000,Q29,3
31258,food & beverage,middle,50-250,Q29,2
31259,clothing & footware,middle,10000-inf,Q29,3
31260,"is not clothing & footwear, is not health & be...","is not luxury, is not upper, is not middle",500-1000,Q29,3


In [108]:
df_stats = df_stats.groupby(list(demo_json.keys()) + ["Q"]).agg(['count', 'mean', 'std']).reset_index()
df_stats.columns = ["sector","price_position","worldwide_store_count","Q","count","mean","std"]

In [109]:
len(df_stats[df_stats["count"] == 1])

1624

In [112]:
mask_c = df_stats["count"] > 1
df_stats[mask_c].describe()

,count,mean,std
count,4582.000000,4582.000000,4582.000000
mean,6.468354,2.678930,0.224246
std,5.798571,1.415068,0.289414
min,2.000000,1.000000,0.000000
25%,3.000000,1.072802,0.000000
50%,5.000000,2.400000,0.000000
75%,8.000000,4.000000,0.500000
max,40.000000,5.000000,2.121320


In [117]:
mask_s = df_stats["std"] == 0
df_stats[mask_c & mask_s].describe()

,count,mean,std
count,2662.000000,2662.000000,2662.0
mean,5.526296,2.362509,0.0
std,5.004682,1.473195,0.0
min,2.000000,1.000000,0.0
25%,3.000000,1.000000,0.0
50%,4.000000,2.000000,0.0
75%,6.000000,4.000000,0.0
max,40.000000,5.000000,0.0


In [121]:
mask_s = df_stats["std"] == 0
df_stats[mask_c & ~mask_s].describe()

,count,mean,std
count,1920.000000,1920.000000,1920.000000
mean,7.774479,3.117633,0.535153
std,6.526546,1.199672,0.182980
min,2.000000,1.025000,0.158114
25%,4.000000,1.960737,0.447214
50%,6.000000,3.333333,0.516398
75%,9.000000,4.250000,0.577350
max,40.000000,4.961538,2.121320
